# NISQ Experiment

In [6]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [7]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import random_unitary, Operator

In [8]:
#qk.IBMQ.save_account("a9ec3c697bede976217e9ff6c5c0f19a2511ea34a225bbe10f0c74bda39de845d16fa3f7c30b47c5714e2b695aded3cfd474e7c1b33056014fab89301b83d724", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibm_oslo")

#backend = AerSimulator()

ibmqfactory.load_account:WARNING:2022-10-18 22:06:21,511: Credentials are already in use. The existing account in the session will be replaced.


In [9]:
def execute_and_collect(circuit_list, backend, file_name, shots=20000):
    N = len(circuit_list)
    num_batches = (N+500-1)//500
    circuit_batch_list = [circuit_list[500*i: 500*(i+1)] for i in range(num_batches)]
    counts_list = []
    for i, circuit_batch in enumerate(tqdm(circuit_batch_list)):
        num_parcels = (len(circuit_batch) + 100 - 1)//100
        circuit_parcel_list = [circuit_batch[100*j: 100*(j+1)] for j in range(num_parcels)]
        job_list = []
        
        for circuit_parcel in circuit_parcel_list:
            trans_circ_list = qk.transpile(circuit_parcel, 
                                           backend, 
                                           optimization_level = 0, 
                                           seed_transpiler=42, 
                                           scheduling_method = 'asap') 
            job = backend.run(trans_circ_list, shots = shots)
            job_list.append(job)
        
        result_list = []
        for job in tqdm(job_list):
            result_list.append(job.result())
            
        for result, circuit_parcel in zip(result_list, circuit_parcel_list):
            counts_list.extend([result.get_counts(circuit) for circuit in circuit_parcel])
        
        pickle.dump(counts_list, open(f"../../data/{file_name}_{i}.p", "wb"))   
        
    return counts_list  

## Variational Circuit

## One Layer

In [35]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 1).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)

In [36]:
print(circuit_target)

      ┌────────────┐ ┌─────────────┐     ┌───┐
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├─────┤ X ├
      ├────────────┤ ├─────────────┤┌───┐└─┬─┘
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├──■──
     ┌┴────────────┴┐├─────────────┤└─┬─┘     
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■───────
     └──────────────┘└─────────────┘          


In [33]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Two Layers

In [38]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 2).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)

In [39]:
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐     ┌───┐
«q_0: ─┤ Rz(2.9525) ├─────┤ X ├
«     ┌┴────────────┤┌───┐└─┬─┘
«q_1: ┤ Rz(-3.0123) ├┤ X ├──■──
«     └─────────────┘└─┬─┘     
«q_2: ─────────────────■───────
«                              


In [40]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_2')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_2.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Three Layers

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 3).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3.p", "wb"))

## Four Layers

In [43]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 4).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [44]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_4')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_4.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Six Layers

In [6]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 6).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [7]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_6')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_6.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Nine Layers

In [8]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [13]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_12')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_12.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Twelve Layers

In [14]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 12, 0, 12).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [15]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_12')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_12.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## L2

In [7]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 6, 3, 6).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐┌──────────────┐          ┌───┐       ┌────────────┐ »
q_0: ┤ Ry(0.70279) ├┤ Rz(-0.83967) ├──────────┤ X ├───────┤ Ry(-1.887) ├─»
     ├─────────────┤└┬────────────┬┘┌───┐     └─┬─┘       ├────────────┤ »
q_1: ┤ Ry(-1.3117) ├─┤ Rz(-1.306) ├─┤ X ├───────■─────────┤ Ry(1.7918) ├─»
     ├─────────────┤┌┴────────────┤ └─┬─┘┌──────────────┐┌┴────────────┴┐»
q_2: ┤ Ry(-0.4276) ├┤ Rz(-2.2651) ├───■──┤ Ry(-0.27602) ├┤ Rz(0.089438) ├»
     └─────────────┘└─────────────┘      └──────────────┘└──────────────┘»
«     ┌─────────────┐          ┌───┐     ┌─────────────┐┌────────────┐     »
«q_0: ┤ Rz(-2.8497) ├──────────┤ X ├─────┤ Ry(-2.7329) ├┤ Rz(1.9377) ├─────»
«     ├─────────────┤┌───┐     └─┬─┘     ├─────────────┤├────────────┤┌───┐»
«q_1: ┤ Rz(0.58066) ├┤ X ├───────■───────┤ Ry(-2.0702) ├┤ Rz(2.9257) ├┤ X ├»
«     └─────────────┘└─┬─┘┌─────────────┐└┬────────────┤└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(0.67572) ├─┤ Rz(2.8204) ├────────────────■──»
«            

In [8]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3_2')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3_2.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## L3

In [8]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 9, 6, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐┌───────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.1576) ├┤ Rz(-0.030304) ├──────────┤ X ├─────┤ Ry(-1.5156) ├»
     ┌┴────────────┤└┬─────────────┬┘┌───┐     └─┬─┘     └┬────────────┤»
q_1: ┤ Ry(-2.5279) ├─┤ Rz(-2.3748) ├─┤ X ├───────■────────┤ Ry(2.5718) ├»
     ├─────────────┤ ├─────────────┴┐└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-1.2276) ├─┤ Rz(-0.37603) ├──■──┤ Ry(-2.9255) ├─┤ Rz(1.0212) ├»
     └─────────────┘ └──────────────┘     └─────────────┘ └────────────┘»
«     ┌─────────────┐          ┌───┐      ┌────────────┐┌────────────┐     »
«q_0: ┤ Rz(0.12609) ├──────────┤ X ├──────┤ Ry(2.9505) ├┤ Rz(2.4808) ├─────»
«     ├─────────────┤┌───┐     └─┬─┘     ┌┴────────────┤├────────────┤┌───┐»
«q_1: ┤ Rz(-1.1831) ├┤ X ├───────■───────┤ Ry(-1.9801) ├┤ Rz(2.7615) ├┤ X ├»
«     └─────────────┘└─┬─┘┌─────────────┐└┬────────────┤└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(0.29349) ├─┤ Rz(1.7287) ├────────────────■──»
«                   

In [12]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3_3')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3_3.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## L4

In [16]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 12, 9, 12).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐┌─────────────┐          ┌───┐       ┌────────────┐ »
q_0: ┤ Ry(-2.5856) ├┤ Rz(-1.0975) ├──────────┤ X ├───────┤ Ry(2.0655) ├─»
     └┬────────────┤├─────────────┤┌───┐     └─┬─┘      ┌┴────────────┤ »
q_1: ─┤ Ry(2.6507) ├┤ Rz(-2.8574) ├┤ X ├───────■────────┤ Ry(-1.4367) ├─»
     ┌┴────────────┤├─────────────┤└─┬─┘┌──────────────┐├─────────────┴┐»
q_2: ┤ Ry(0.61512) ├┤ Rz(-1.9102) ├──■──┤ Ry(-0.69946) ├┤ Rz(-0.90005) ├»
     └─────────────┘└─────────────┘     └──────────────┘└──────────────┘»
«     ┌─────────────┐          ┌───┐     ┌─────────────┐┌────────────┐     »
«q_0: ┤ Rz(0.26827) ├──────────┤ X ├─────┤ Ry(-2.6732) ├┤ Rz(-1.893) ├─────»
«     ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤├────────────┤┌───┐»
«q_1: ┤ Rz(-1.3764) ├┤ X ├───────■────────┤ Ry(1.8988) ├┤ Rz(1.7106) ├┤ X ├»
«     └─────────────┘└─┬─┘┌─────────────┐ ├────────────┤└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(-2.2561) ├─┤ Rz(3.0592) ├────────────────■──»
«                   

In [17]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_3_4')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_3_4.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
n = 3
d = 2**n

np.random.seed(43)
random.seed(43)

circuit_target = variational_circuit(n, 9, 0, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐ ┌────────────┐         ┌───┐     ┌─────────────┐»
q_0: ┤ Ry(-2.3035) ├─┤ Rz(2.2565) ├─────────┤ X ├─────┤ Ry(-2.9593) ├»
     ├─────────────┤┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤»
q_1: ┤ Ry(0.68529) ├┤ Rz(-1.0861) ├┤ X ├──────■───────┤ Ry(0.25863) ├»
     ├─────────────┤├─────────────┤└─┬─┘┌────────────┐└┬────────────┤»
q_2: ┤ Ry(-2.4187) ├┤ Rz(-1.6299) ├──■──┤ Ry(1.0436) ├─┤ Rz(1.4687) ├»
     └─────────────┘└─────────────┘     └────────────┘ └────────────┘»
«      ┌────────────┐          ┌───┐      ┌────────────┐┌─────────────┐      »
«q_0: ─┤ Rz(1.8978) ├──────────┤ X ├──────┤ Ry(2.3037) ├┤ Rz(-1.1555) ├──────»
«     ┌┴────────────┴┐┌───┐    └─┬─┘     ┌┴────────────┤├─────────────┴┐┌───┐»
«q_1: ┤ Rz(-0.66005) ├┤ X ├──────■───────┤ Ry(-2.7842) ├┤ Rz(-0.59697) ├┤ X ├»
«     └──────────────┘└─┬─┘┌────────────┐├─────────────┤└──────────────┘└─┬─┘»
«q_2: ──────────────────■──┤ Ry(-1.543) ├┤ Rz(-1.7528) ├──────────────────■──»
«                          └─

In [6]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_9_2')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_9_2.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
n = 3
d = 2**n

np.random.seed(44)
random.seed(44)

circuit_target = variational_circuit(n, 9, 0, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

      ┌────────────┐┌─────────────┐           ┌───┐     ┌──────────────┐»
q_0: ─┤ Ry(1.5371) ├┤ Rz(0.68636) ├───────────┤ X ├─────┤ Ry(0.062219) ├»
     ┌┴────────────┤├─────────────┴┐┌───┐     └─┬─┘     ├──────────────┤»
q_1: ┤ Ry(-2.4831) ├┤ Rz(-0.88398) ├┤ X ├───────■───────┤ Ry(-0.57131) ├»
     └┬────────────┤├─────────────┬┘└─┬─┘┌─────────────┐└┬────────────┬┘»
q_2: ─┤ Ry(2.1039) ├┤ Rz(-0.8765) ├───■──┤ Ry(-0.6674) ├─┤ Rz(1.3204) ├─»
      └────────────┘└─────────────┘      └─────────────┘ └────────────┘ »
«     ┌──────────────┐          ┌───┐      ┌─────────────┐┌────────────┐     »
«q_0: ┤ Rz(-0.27256) ├──────────┤ X ├──────┤ Ry(-1.7725) ├┤ Rz(2.3991) ├─────»
«     └┬────────────┬┘┌───┐     └─┬─┘      ├─────────────┤├────────────┤┌───┐»
«q_1: ─┤ Rz(2.8936) ├─┤ X ├───────■────────┤ Ry(-2.4287) ├┤ Rz(2.7857) ├┤ X ├»
«      └────────────┘ └─┬─┘┌──────────────┐└┬────────────┤└────────────┘└─┬─┘»
«q_2: ──────────────────■──┤ Ry(-0.45458) ├─┤ Rz(2.8744) ├────────────────■──»
«       

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_9_3')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_9_3.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
n = 3
d = 2**n

np.random.seed(45)
random.seed(45)

circuit_target = variational_circuit(n, 9, 0, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐┌──────────────┐          ┌───┐     ┌─────────────┐»
q_0: ┤ Ry(-1.3732) ├┤ Rz(-0.17085) ├──────────┤ X ├─────┤ Ry(-2.4131) ├»
     └┬────────────┤├──────────────┤┌───┐     └─┬─┘     ├─────────────┤»
q_1: ─┤ Ry(0.3113) ├┤ Rz(-0.34891) ├┤ X ├───────■───────┤ Ry(-2.1154) ├»
      ├────────────┤└┬────────────┬┘└─┬─┘┌─────────────┐├─────────────┤»
q_2: ─┤ Ry(3.0725) ├─┤ Rz(-2.656) ├───■──┤ Ry(-2.8367) ├┤ Rz(0.80043) ├»
      └────────────┘ └────────────┘      └─────────────┘└─────────────┘»
«     ┌─────────────┐         ┌───┐     ┌─────────────┐ ┌────────────┐     »
«q_0: ┤ Rz(0.94312) ├─────────┤ X ├─────┤ Ry(0.74327) ├─┤ Rz(1.0874) ├─────»
«     └┬───────────┬┘┌───┐    └─┬─┘     ├─────────────┴┐├────────────┤┌───┐»
«q_1: ─┤ Rz(2.238) ├─┤ X ├──────■───────┤ Ry(-0.18629) ├┤ Rz(2.9908) ├┤ X ├»
«      └───────────┘ └─┬─┘┌────────────┐├─────────────┬┘└────────────┘└─┬─┘»
«q_2: ─────────────────■──┤ Ry(3.0833) ├┤ Rz(-1.3655) ├─────────────────■──»
«                         └

In [7]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_9_4')
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_9_4.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
n = 3
d = 2**n

np.random.seed(46)
random.seed(46)

circuit_target = variational_circuit(n, 9, 0, 9).reverse_bits()

N = 1000-6**n
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

inputs_spam, circuit_spam = generate_pauliInput_circuits(n)

circuit_list.extend(circuit_spam)
print(circuit_target)

     ┌─────────────┐ ┌────────────┐          ┌───┐      ┌────────────┐ »
q_0: ┤ Ry(-1.5768) ├─┤ Rz(2.7472) ├──────────┤ X ├──────┤ Ry(2.5932) ├─»
     ├─────────────┤┌┴────────────┤┌───┐     └─┬─┘     ┌┴────────────┴┐»
q_1: ┤ Ry(0.84719) ├┤ Rz(-1.1745) ├┤ X ├───────■───────┤ Ry(-0.37154) ├»
     └┬────────────┤└┬────────────┤└─┬─┘┌─────────────┐├──────────────┤»
q_2: ─┤ Ry(1.7834) ├─┤ Rz(1.6215) ├──■──┤ Ry(-2.8723) ├┤ Rz(-0.28272) ├»
      └────────────┘ └────────────┘     └─────────────┘└──────────────┘»
«     ┌─────────────┐           ┌───┐       ┌────────────┐ ┌────────────┐     »
«q_0: ┤ Rz(-2.6081) ├───────────┤ X ├───────┤ Ry(2.3064) ├─┤ Rz(1.8982) ├─────»
«     ├─────────────┴┐┌───┐     └─┬─┘       ├────────────┤┌┴────────────┤┌───┐»
«q_1: ┤ Rz(0.049853) ├┤ X ├───────■─────────┤ Ry(1.5439) ├┤ Rz(-2.4877) ├┤ X ├»
«     └──────────────┘└─┬─┘┌──────────────┐┌┴────────────┤└─────────────┘└─┬─┘»
«q_2: ──────────────────■──┤ Ry(-0.46276) ├┤ Rz(-1.1079) ├─────────────────■──»
«        

In [10]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_9_7', shots=2*1024)
data = [input_map_list, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_9_7.p", "wb"))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Greedy Sampling

In [5]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n, 9, 0, 9).reverse_bits()
print(circuit_target)

N = 1800
input_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

      ┌────────────┐ ┌─────────────┐     ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├─────┤ X ├──■──┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐└─┬─┘  │  └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├──■────┼───┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘     ┌─┴─┐┌┴────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■───────┤ X ├┤ Ry(-2.7766) ├»
     └──────────────┘└─────────────┘          └───┘└─────────────┘»
«      ┌────────────┐     ┌───┐     ┌─────────────┐┌─────────────┐     ┌───┐»
«q_0: ─┤ Rz(2.9525) ├─────┤ X ├──■──┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────┤ X ├»
«     ┌┴────────────┤┌───┐└─┬─┘  │  ├─────────────┤└┬───────────┬┘┌───┐└─┬─┘»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──■────┼──┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├──■──»
«     └┬────────────┤└─┬─┘     ┌─┴─┐└┬────────────┤┌┴───────────┴┐└─┬─┘     »
«q_2: ─┤ Rz(1.3074) ├──■───────┤ X ├─┤ Ry(2.0888) ├┤ Rz(-1.9892) ├──■───────»
«      └────────────┘          └───┘ └────────────┘└────

In [6]:
counts_map = execute_and_collect(circuit_list_map, backend, 'threeQubits_variational_greedy_map', shots = 1000)
counts_spam = execute_and_collect(circuit_list_spam, backend, 'threeQubits_variational_greedy_spam', shots = 10000)
data = [input_map, counts_map, inputs_spam, counts_spam]
pickle.dump(data, open("../../data/threeQubits_variational_greedy.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

## Delay Circuit

### No Delay

In [6]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_noDelay_manila.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 2.5 mu

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(2.5, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_2point5usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_2point5usDelay_manila.p", "wb"))

### 5 mu

In [18]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(5, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_5usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_5usDelay_manila.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 6 mu

In [10]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(6, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [11]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_6usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_6usDelay_manila.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 7 mu

In [12]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(7, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [13]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_7usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_7usDelay_manila.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 8 mu

In [14]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(8, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [15]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_8usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_8usDelay_manila.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 9 mu

In [16]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(9, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

      ┌────────────┐ ┌─────────────┐          ┌───┐     ┌─────────────┐»
q_0: ─┤ Ry(1.4577) ├─┤ Rz(-2.1615) ├──────────┤ X ├─────┤ Ry(0.63532) ├»
      ├────────────┤ ├─────────────┤┌───┐     └─┬─┘     └┬────────────┤»
q_1: ─┤ Ry(2.8319) ├─┤ Rz(-2.1613) ├┤ X ├───────■────────┤ Ry(2.3008) ├»
     ┌┴────────────┴┐├─────────────┤└─┬─┘┌─────────────┐ ├────────────┤»
q_2: ┤ Ry(-0.78829) ├┤ Rz(0.61989) ├──■──┤ Ry(-2.7766) ├─┤ Rz(1.3074) ├»
     └──────────────┘└─────────────┘     └─────────────┘ └────────────┘»
«      ┌────────────┐         ┌───┐     ┌─────────────┐┌─────────────┐     »
«q_0: ─┤ Rz(2.9525) ├─────────┤ X ├─────┤ Ry(-1.9992) ├┤ Rz(0.15555) ├─────»
«     ┌┴────────────┤┌───┐    └─┬─┘     ├─────────────┤└┬───────────┬┘┌───┐»
«q_1: ┤ Rz(-3.0123) ├┤ X ├──────■───────┤ Ry(-1.8074) ├─┤ Rz(-1.23) ├─┤ X ├»
«     └─────────────┘└─┬─┘┌────────────┐├─────────────┤ └───────────┘ └─┬─┘»
«q_2: ─────────────────■──┤ Ry(2.0888) ├┤ Rz(-1.9892) ├─────────────────■──»
«                         └

In [17]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_9usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_9usDelay_manila.p", "wb"))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 10 mu

In [ ]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target.delay(10, unit="us")
print(circuit_target)

N = 2000 - 6**n
inputs_map, circuit_list_map = generate_pauli_circuits(n, circuit_target, N)
inputs_spam, circuit_list_spam = generate_pauliInput_circuits(n)

circuit_list = circuit_list_map + circuit_list_spam

In [ ]:
counts_list = execute_and_collect(circuit_list, backend, 'threeQubits_variational_10usDelay_manila', shots = 5000)
data = [inputs_map, inputs_spam, counts_list]
pickle.dump(data, open("../../data/threeQubits_variational_10usDelay_manila.p", "wb"))